# Lab 9 - DEFI(Uniswap V3)

## 5. Add Liquidity

In [ ]:
artifacts = {
    WETH9: require("./build/WETH9.json"),
    UniswapV3Factory: require("./build/UniswapV3Factory.json"),
    UniswapV3Pool: require('./build/UniswapV3Pool.json'),
    NFTDescriptor: require("./build/NFTDescriptor.json"),
    NonfungibleTokenPositionDescriptor: require("./build/NonfungibleTokenPositionDescriptor.json"),
    NonfungiblePositionManager: require("./build/NonfungiblePositionManager.json"),
    ERC20: require("./artifacts/contracts/TutorialERC20.sol/TutorialERC20.json")
};

In [ ]:
contracts = {
  WETH: '0x78E6935fD0D9b57B1Bc20204BA769bCEd249012E',
  UNISWAPV3_FACTORY: '0xfF2868E36dEfdD0180DFaFd4c234B9c2588a1887',
  NONFUNGIBLE_POSITION_MANAGER: '0xFB04DA8DCc30fbD00FD953ADE7414953Ccc96145',
  TOKEN_0: '0x3cE05DC47cCBd7c62E6dC6f1d6a08493B18B7992',
  TOKEN_1: '0xff64fbE6824cC5CA450FD4Ca22c249a6d375028f',
  POOL: '0x9fE073f12AD63A8bb14B5a40F452C45Aa1a7426D'
};

In [ ]:
new Promise(async (res,rej)=>{

    const {Contract, ethers} = require('ethers');
    const provider = new ethers.providers.JsonRpcProvider("http://localhost:8545");
    const account = provider.getSigner();

    // Configure Pool
    poolContract = new Contract(
        contracts.POOL,
        artifacts.UniswapV3Pool.abi, 
        provider);

    const [tickSpacing, fee, liquidity, slot0] = await Promise.all([
        poolContract.tickSpacing(),
        poolContract.fee(),
        poolContract.liquidity(),
        poolContract.slot0(),
    ]);
    sqrtPriceX96 = slot0[0];
    tick = slot0[1];

    poolData = {
        tickSpacing: tickSpacing,
        fee: fee,
        liquidity: liquidity,
        sqrtPriceX96: slot0[0],
        tick: slot0[1],
    };


    const { Token } = require("@uniswap/sdk-core");
    token0 = new Token(31337, contracts.TOKEN_0, 18, "TutA", "TutA");
    token1 = new Token(31337, contracts.TOKEN_1, 18, "TutB", "TutB");

    const { Pool, Position, nearestUsableTick } = require("@uniswap/v3-sdk");
    let configuredPool = new Pool(
        token0,
        token1,
        poolData.fee,
        poolData.sqrtPriceX96.toString(),
        poolData.liquidity.toString(),
        poolData.tick
    );    

    // Define Position from configured Pool

    let position = new Position({
        pool: configuredPool,
        liquidity: ethers.utils.parseEther("1"),
        tickLower:
            nearestUsableTick(poolData.tick, poolData.tickSpacing) -
            poolData.tickSpacing * 2,
        tickUpper:
            nearestUsableTick(poolData.tick, poolData.tickSpacing) +
            poolData.tickSpacing * 2,
    });

    // Mint Position

    let { amount0: amount0Desired, amount1: amount1Desired } = position.mintAmounts;

    params = {
        token0: token0.address,
        token1: token1.address,
        fee: poolData.fee,
        tickLower:
            nearestUsableTick(poolData.tick, poolData.tickSpacing) -
            poolData.tickSpacing * 2,
        tickUpper:
            nearestUsableTick(poolData.tick, poolData.tickSpacing) +
            poolData.tickSpacing * 2,
        amount0Desired: amount0Desired.toString(),
        amount1Desired: amount1Desired.toString(),
        amount0Min: 0,
        amount1Min: 0,
        recipient: token1.address,
        deadline: Math.floor(Date.now() / 1000) + 60 * 10,
    };

    const nonfungiblePositionManager = new Contract(contracts.NONFUNGIBLE_POSITION_MANAGER,artifacts.NonfungiblePositionManager.abi,provider)

    const tx = await nonfungiblePositionManager
        .connect(account)
        .mint(params, { gasLimit: "1000000" });
    const receipt = await tx.wait();
    console.log(receipt);
    
})